## 데이터 및 라이브러리 로드

In [ ]:
import os
import random
import pickle
import numpy as np
import pandas as pd
import gc
import tensorflow as tf
from custom_metric import *

SEED = 30
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
with open('D:/inputs/24/train.pickle', 'rb') as f:
    train = pickle.load(f)
with open('D:/inputs/24/train_y.pickle', 'rb') as f:
    train_y = pickle.load(f)
    train_y = train_y.reshape(train_y.shape[0], 40, 40, 1)


with open('D:/inputs/24/test.pickle', 'rb') as f:
    test = pickle.load(f)
LEN_TRAIN = train.shape[0]

In [ ]:
train.shape

In [ ]:
display(
    train.shape,
    train_y.shape,
    test.shape
)

In [ ]:
# 조건에 맞는 사진 개수 세는 알고리즘.
# from tqdm import tqdm
# cnt = 0
# for i in tqdm(train_y):
#     for ii in i:
#         if (ii == 0).sum() != 40:
#             cnt += 1
# cnt

## 9번 channel 가공 - 카테고리형 피쳐

In [ ]:
# train = np.delete(train, 9, axis=3)
# test = np.delete(test, 9, axis=3)

In [ ]:
# with open('D:/inputs/24/rot_train.pickle', 'rb') as f:
#     rot_train = pickle.load(f)
# with open('D:/inputs/24/rot_train_y.pickle', 'rb') as f:
#     rot_train_y = pickle.load(f)
#     rot_train_y = rot_train_y.reshape(rot_train_y.shape[0], 40, 40, 1)
# LEN_TRAIN = train.shape[0] + rot_train.shape[0]

# train = np.vstack([train, rot_train])
# train_y = np.vstack([train_y, rot_train_y])

# del rot_train
# del rot_train_y
# gc.collect()

## all_np로 결합 후 피쳐 생성

In [ ]:
from features import rotation

train, test = rotation(train, test)

In [ ]:
TR_SHAPE_0, TR_SHAPE_1, TR_SHAPE_2, TR_SHAPE_3 = train.shape
TE_SHAPE_0, TE_SHAPE_1, TE_SHAPE_2, TE_SHAPE_3 = test.shape

## all_np로 결합 후 scaling

In [ ]:
from preprocess import scaling

trian, test = scaling(train, test, 'ss') # 3번째 인자 'rs', 'ms', 'ss' 

In [ ]:
gc.collect()

In [ ]:
from models import build_model, build_model2

In [ ]:
input_layer = Input((40, 40, train.shape[3]))
output_layer = build_model2(input_layer, 32)
model = Model(input_layer, output_layer)

# model.compile(loss="mae", optimizer="adam", metrics=[maeOverFscore_keras, fscore_keras])
model.compile(loss="mae", optimizer="adam", metrics=["mae"])

scores = []

In [ ]:
model = tf.keras.models.load_model('epochs_8.h5')
model.summary()

In [ ]:
EPOCHS = 30
for _ in range(5):
    for i in range(EPOCHS):
        print(f"epochs: {i+1}")
        model_history = model.fit(train, train_y, epochs = 1, verbose=1, batch_size = 200)
    score = maeOverFscore(train_y, model.predict(train))
    print('mae / fscore: ' + str(score) + '\n')
    scores.append(score)
    print(scores)

## mae loss graph

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (10,5))
plt.plot(scores)


## 모델 저장 및 submission 생성

In [ ]:
model_name = input('모델 입력을 입력하세요! : ')
model.save(model_name)

In [ ]:
model.save('epochs600.h5')

In [ ]:
model.save('epochs.h5')

In [ ]:
pred = model.predict(test)
submission = pd.read_csv('inputs/sample_submission.csv')
submission.iloc[:,1:] = pred.reshape(-1, 1600)
submission.to_csv('1.374.csv', index = False)

## 해야할 일

1. ~~스케일링(Scaling) - Robust, log, Standard Scaler  ~~
2. ~~위도 경도 피쳐를 가지고 위도+경도, 위도-경도 피쳐 만들기(45도 회전변환)~~: 30, 60도 변환 추가
3. 현재 custom metric이 작동하지 않으므로, 이 또한 수정해야함.
4. tree모델 만들고, 앙상블
5. ~~target 값에 log 취하기~~: 점수 안오름
6. 순서가 바뀌긴 했으나, EDA 작업하자.
7. 강수량 타일이 10개 미만인 것의 평균, 20개미만의 평균.... 50개 미만의 평균... 알아보자
8. ~~augumentation 사용해서 강수량이 일정수준 이상인 사진 augementation 하자~~ - 다만, regression인 만큼 가장자리 어떻게 처리할지 생각해보자. - 정사각형 사진이기에 90도 돌린사진, 180도 돌린사진, 270도 돌린사진 이렇게만 해도 데이터가 증식될거같음 -  
**그런데!!!!!** 생각해봐야할것이.....  
위도, 경도 등 여러 요소가 존재할텐데 돌려버리면 원래는 비 안오는거를 비온다고 볼 수도 있을 듯 싶다. 쉽게 예를 들면 I 모양일때는 비가 오고, ㅡ 모양일때는 비가안오는데, 돌리는 바람에 안오는걸 온다고 할 수도 있을 것 같다. - 이건 테스트해볼수밖에 없을 듯 싶다. 
9. pretrained 모델 활용방안 생각하기 - 최소 사진크기가 존재하는데 우리는 40x40이라 작은편.. 이걸 40x40을 이어 붙여서 200x200으로 만들어서 넣으면 될거같은데 시도해보자.
10. 카테고리형 변수 원핫인코딩

| 리더보드(5부터) | mae / fscore | mae | 비고 |
|---|:---:|---:|---:|
| 1.93737 | 1.8744 | 0.2451 |  |
| 1.75177 | 1.7016 | 0.2384 |  |
| 1.61793 | 1.4234 | 0.0832 | 리더보드랑 metric 점수랑 차이 많이 남. 해결해야함. |